# Patch Basics - Exercise and solution

April 14, 2025


In [ ]:
%%capture

# First ensure DASCore is installed. If not, install and restart the kernel.
try:
    import dascore as dc
except ImportError:
    !pip install dascore
    !pip install ipympl
    # resetart kernel
    import IPython
    IPython.Application.instance().kernel.do_shutdown(True) #automatically restarts kernel


import numpy as np
from rich import print


### **Exercise** (`Patch` Data)

Calculate and print following:

1. The number of samples in the data
2. The minimum and maximum value of the data

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

# The number of samples in the data
num_samples = patch.data.size
print(f"Number of samples: {num_samples}")

# The minimum and maximum value of the data
min_val = patch.data.min()
max_val = patch.data.max()
print(f"Minimum value: {min_val}")
print(f"Maximum value: {max_val}")

### **Exercise** (`Patch` Attrs)
Do the following:

1. Print the `cable_id` from the patch with updated attrs.
2. Create a new patch with a `station name` of "DAS1".

In [ ]:
patch_updated_attrs = patch.update_attrs(
    acquisition_id="experiment_12",
    cable_id="b202393ad",
)

# 1. Print the cable_id from the patch with updated attrs
print(f"Cable ID: {patch_updated_attrs.attrs['cable_id']}")

# 2. Create a new patch with a station name of "DAS1"
patch_with_station = patch.update_attrs(station_name="DAS1")
print(patch_with_station.attrs)

### **Exercise** (`Patch` Coords)

Calculate the and print following parameters:

1. The duration (time) of the patch recording using the time coordinate.
2. Reset the start of the time coordiante (e.g., 08:00, April 14, 2025).


In [ ]:
import numpy as np
from datetime import datetime

# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

# 1. Calculate the duration of the patch recording
time_coord = patch.get_coord("time")
time_start = time_coord.min()
time_end = time_coord.max()
duration = time_end - time_start
print(f"Duration: {duration}")

# 2. Reset the start of the time coordiante (e.g., 08:00, April 14, 2025)

# New start time
new_start = np.datetime64('2025-04-14T08:00:00')
current_time = patch.get_array("time")

# Calculate time differences from start
time_delta = current_time - current_time[0] 

# Add to new start time
new_time = new_start + time_delta  

# Create new patch with updated time coordinate
patch_new = patch.update_coords(time=new_time)

print(f"New time range: {patch_new.get_coord('time').min()} to {patch_new.get_coord('time').max()}")

### **Exercise** (`Patch` Select)
Remove the first 10 spatial channels then the last .05 seconds. 

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

# First, remove the first 10 spatial channels
patch_trimmed_distance = patch.select(distance=(10, None), samples=True)

# Then, remove the last 0.05 seconds
patch_final_trim = patch_trimmed_distance.select(time=(None, -0.05), relative=True)

# Verify the trimming worked
print(f"Original patch shape: {patch.data.shape}")
print(f"Trimmed patch shape: {patch_final_trim.data.shape}")


### **Exercise** (`Patch` Processing 1)
Plot each of the filtered patches above. Which filtering technique did the best at accentuating the event signal? 

In [ ]:
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

patch_bp = patch.pass_filter(time=(100, 300))  # apply a 100Hz to 300Hz highpass
patch_lp = patch.pass_filter(time=(..., 300))  # apply a 300Hz lowpass
patch_hp = patch.pass_filter(time=(50, ...))  # apply a 50Hz highpass

# Create a figure with 3 subplots
fig = plt.figure(figsize=(10, 16))
gs = GridSpec(3, 1, figure=fig)

# Plot the bandpass filtered patch
ax1 = fig.add_subplot(gs[0, 0])
patch_bp.viz.waterfall(ax=ax1, scale=0.1)
ax1.set_title('Bandpass Filter (100-300 Hz)')

# Plot the lowpass filtered patch
ax2 = fig.add_subplot(gs[1, 0])
patch_lp.viz.waterfall(ax=ax2, scale=0.1)
ax2.set_title('Lowpass Filter (<300 Hz)')

# Plot the highpass filtered patch
ax3 = fig.add_subplot(gs[2, 0])
patch_hp.viz.waterfall(ax=ax3, scale=0.1)
ax3.set_title('Highpass Filter (>50 Hz)')

plt.tight_layout()
plt.show()

### **Exercise** (`Patch` Processing 2)

Find some methods in the [processing module documentation](https://dascore.org/api/dascore/proc.html) and apply them to the event patch. Visualize the results. 

In [ ]:
# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")

# 1. Try the normalize method to normalize trace amplitudes
patch_norm = patch.normalize("time",norm='max')

# 2. Try the taper method to apply a taper to the edges
# Apply an Hanning taper to 5% of each end for time dimension.
patch_taper1 = patch.taper(time=0.05, window_type="hann")
# Apply a triangular taper to 10% of the start of the distance dimension.
patch_taper2 = patch.taper(distance=(0.10, None), window_type='triang')

# 3. Try the differentiate method (velocity to acceleration)
patch_diff = patch.differentiate("time")

# Visualize the results
fig = plt.figure(figsize=(10, 16))
gs = GridSpec(3, 1, figure=fig)

# Plot normalized patch
ax1 = fig.add_subplot(gs[0, 0])
patch_norm.viz.waterfall(ax=ax1, scale=1)
ax1.set_title('Normalized Patch')

# Plot tapered patch
ax2 = fig.add_subplot(gs[1, 0])
patch_taper1.viz.waterfall(ax=ax2, scale=0.1)
ax2.set_title('Tapered Patch')

# Plot differentiated patch
ax3 = fig.add_subplot(gs[2, 0])
patch_diff.viz.waterfall(ax=ax3, scale=0.1)
ax3.set_title('Differentiated Patch')

plt.tight_layout()
plt.show()

### **Exercise** (`Patch` Transformation)
Do the following to the patch above:

1. Perform a real discrete Fourier transform along the time axis
2. Get the amplitude spectra
3. Take the mean along the distance dimension
4. Select frequencies (`ft_time` dimension) less than 500 Hz
5. Plot using matplotlib and the `ft_time` dimension's values


In [ ]:
import matplotlib.pyplot as plt

# The get_example_patch function is useful for loading example/test patches.
patch = dc.get_example_patch("example_event_1")


# 1. Perform a real discrete Fourier transform along the time axis
fft_patch = patch.dft("time", real=True)

# 2. Get the amplitude spectra
amp_spectra = fft_patch.abs()

# 3. Take the mean along the distance dimension
mean_spectra = amp_spectra.mean("distance")

# 4. Select frequencies less than 500 Hz
low_freq_spectra = mean_spectra.select(ft_time=(0, 500))

# 5. Plot using matplotlib
plt.figure(figsize=(10, 6))
freq = low_freq_spectra.get_array("ft_time")

# We need to squeeze/flatten the array to make it 1D for plotting
data_to_plot = low_freq_spectra.data.squeeze()  # Remove singleton dimensions

plt.plot(freq, data_to_plot)
plt.xlabel('Frequency (Hz)')
plt.ylabel('Amplitude')
plt.title('Mean Amplitude Spectrum (<500 Hz)')
plt.grid(linewidth=0.5,linestyle='dashed')
plt.show()